In [0]:
base = "/Volumes/advisory_demo_catalog/ai_bi_reporting/anon_files"
df_acct = spark.read.option("header","true").csv(f"{base}/ANON BST ACCT*.csv")
df_opp = spark.read.option("header","true").csv(f"{base}/ANON BST OPPS*.csv")
df_cust = spark.read.option("header","true").csv(f"{base}/ANON  Cust_*.csv")

df_acct.write.mode("overwrite").saveAsTable("advisory_demo_catalog.ai_bi_reporting.anon_bst_acct_bz")
df_opp.write.mode("overwrite").saveAsTable("advisory_demo_catalog.ai_bi_reporting.anon_bst_opp_bz")
df_cust.write.mode("overwrite").saveAsTable("advisory_demo_catalog.ai_bi_reporting.anon_cust_intelligence_opportunity_bz")